In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
import re
import random
%run ../py/sql_utlis.py
%run ../py/html_analysis.py

ZMQInteractiveShell_obj = get_ipython()
su = SqlUtilities()
_, CURSOR = su.get_jh_conn_cursor()
ha = HeaderAnalysis()
HEADER_PATTERN_DICT = s.load_object('HEADER_PATTERN_DICT')

Wall time: 3.91 s


In [40]:

# SET other subtypes as 0; assume 0 rows affected if primary and secondary columns are the same
subtypes_list = ['is_task_scope', 'is_minimum_qualification', 'is_preferred_qualification',
                 'is_legal_notification', 'is_job_title', 'is_office_location', 'is_job_duration',
                 'is_supplemental_pay', 'is_educational_requirement', 'is_interview_procedure',
                 'is_corporate_scope', 'is_posting_date', 'is_other']
for primary_column in subtypes_list:
    for secondary_column in subtypes_list:
        CURSOR.execute(su.set_secondary_column0_formatted_sql_str.format(secondary_column, primary_column,
                                                                         secondary_column))

In [10]:

child_strs_set = set()
for file_name, feature_dict_list in HEADER_PATTERN_DICT.items():
    for feature_dict in feature_dict_list:
        if 'is_task_scope' not in feature_dict:
            child_strs_set.add(feature_dict['child_str'])
CHILD_STRS_LIST = list(child_strs_set)
SQL_STR = '''
    SELECT np.[navigable_parent]
    FROM [Jobhunting].[dbo].[NavigableParents] np
    WHERE
        (np.[is_task_scope] IS NULL) AND
        ((np.[is_header] IS NULL) OR
        (np.[is_header] = 0));'''

In [24]:

def get_dictionary_code():
    nonheader_quals_df = pd.DataFrame(su.get_execution_results(CURSOR, SQL_STR, verbose=False))
    mask_series = nonheader_quals_df.navigable_parent.isin(CHILD_STRS_LIST)
    output_str = ''
    child_strs_list = nonheader_quals_df[mask_series].navigable_parent.tolist()
    tag_str = random.choice(child_strs_list)
    output_str += f'\n# Only {len(child_strs_list):,} possible Task Scope non-headers to go\n'
    if "'" in tag_str:
        tag_str = tag_str.replace('"', '\\"')
        output_str += f'tag_str = "{tag_str}"\n'
    else:
        output_str += f"tag_str = '{tag_str}'\n"
    output_str += 'sql_str = r"""\n    UPDATE NavigableParents\n    SET\n        is_header = 0,\n'
    output_str += '        is_task_scope = 0\n    WHERE (navigable_parent LIKE ? ESCAPE \'\\\')"""\n'
    output_str += r"cursor_obj = CURSOR.execute(sql_str, (su.wc_rgx.sub(r'\\\g<1>', tag_str),))"
    output_str += '\nCURSOR.commit()\n\n# If is_header = 0 and is_task_scope = 1 above,'
    output_str += ' this needs to be True\n'
    output_str += "ha.store_true_or_false_dict('O_TS_DICT', tag_str, False)\n"
    output_str += "O_TS_DICT = s.load_object('O_TS_DICT')\nprint(len(O_TS_DICT.keys()))"
    
    return output_str

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_dictionary_code(), replace=True)

In [69]:

# Only 2,326 possible Task Scope non-headers to go
tag_str = '<p>· Experience interpreting and communicating analytic results to analytical and non-analytical business partners</p>'
sql_str = r"""
    UPDATE NavigableParents
    SET
        is_header = 0,
        is_task_scope = 0
    WHERE (navigable_parent LIKE ? ESCAPE '\')"""
cursor_obj = CURSOR.execute(sql_str, (su.wc_rgx.sub(r'\\\g<1>', tag_str),))
CURSOR.commit()

# If is_header = 0 and is_task_scope = 1 above, this needs to be True
ha.store_true_or_false_dict('O_TS_DICT', tag_str, False)
O_TS_DICT = s.load_object('O_TS_DICT')
print(len(O_TS_DICT.keys()))